In [1]:
from __future__ import print_function, division

from keras.datasets import cifar10
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply, GaussianNoise
from keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras import losses
from keras.utils import to_categorical
import keras.backend as K
from PIL import Image
import matplotlib.pyplot as plt
import os
import numpy as np

class SGAN:
    def __init__(self):
        self.img_rows = 32
        self.img_cols = 32
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.num_classes = 10
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(
            loss=['binary_crossentropy', 'categorical_crossentropy'],
            loss_weights=[0.5, 0.5],
            optimizer=optimizer,
            metrics=['accuracy']
        )

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        noise = Input(shape=(100,))
        img = self.generator(noise)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The valid takes generated images as input and determines validity
        valid, _ = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains generator to fool discriminator
        self.combined = Model(noise, valid)
        self.combined.compile(loss=['binary_crossentropy'], optimizer=optimizer)

    def build_generator(self):

        model = Sequential()

        model.add(Dense(128 * 8 * 8, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((8, 8, 128)))
        model.add(BatchNormalization(momentum=0.8))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3, padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(3, kernel_size=3, padding="same"))
        model.add(Activation("tanh"))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        # model.add(ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())

        model.summary()

        img = Input(shape=self.img_shape)

        features = model(img)
        valid = Dense(1, activation="sigmoid")(features)
        label = Dense(self.num_classes+1, activation="softmax")(features)

        return Model(img, [valid, label])

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, y_train), (_, _) = cifar10.load_data()

        # Rescale -1 to 1
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        # X_train = np.expand_dims(X_train, axis=3)
        y_train = y_train.reshape(-1, 1)

        # Class weights:
        # To balance the difference in occurences of digit class labels.
        # 50% of labels that the discriminator trains on are 'fake'.
        # Weight = 1 / frequency
        half_batch = batch_size // 2
        cw1 = {0: 1, 1: 1}
        cw2 = {i: self.num_classes / half_batch for i in range(self.num_classes)}
        cw2[self.num_classes] = 1 / half_batch

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        nb_batches = int(X_train.shape[0] / batch_size)
        global_step = 0

        for epoch in range(epochs):

            for index in range(nb_batches):
                global_step += 1
                imgs = X_train[index * batch_size:(index + 1) * batch_size]
                # Sample noise and generate a batch of new images
                noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
                gen_imgs = self.generator.predict(noise)

                # One-hot encoding of labels
                labels = to_categorical(y_train[index * batch_size:(index + 1) * batch_size], num_classes=self.num_classes + 1)
                fake_labels = to_categorical(np.full((batch_size, 1), self.num_classes),
                                             num_classes=self.num_classes + 1)

                # Train the discriminator
                d_loss_real = self.discriminator.train_on_batch(imgs, [valid, labels], class_weight=[cw1, cw2])
                d_loss_fake = self.discriminator.train_on_batch(gen_imgs, [fake, fake_labels], class_weight=[cw1, cw2])
                d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

                # ---------------------
                #  Train Generator
                # ---------------------

                g_loss = self.combined.train_on_batch(noise, valid, class_weight=[cw1, cw2])

                # Plot the progress
                print("epoch:%d step:%d[D loss: %f, acc: %.2f%%, op_acc: %.2f%%] [G loss: %f]" % (
                    epoch, global_step, d_loss[0], 100 * d_loss[3], 100 * d_loss[4], g_loss))

                # If at save interval => save generated image samples
                if global_step % sample_interval == 0:
                    self.sample_images(epoch, global_step)

    def sample_images(self, epoch, global_step):
        r, c = 10, 10
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        generated_images = self.generator.predict(noise)

        generated_images = np.asarray((generated_images * 127.5 + 127.5).astype(np.uint8))

        def vis_square(data, padsize=1, padval=0):
            # force the number of filters to be square
            n = int(np.ceil(np.sqrt(data.shape[0])))
            padding = ((0, n ** 2 - data.shape[0]), (0, padsize), (0, padsize)) + ((0, 0),) * (data.ndim - 3)
            data = np.pad(data, padding, mode='constant', constant_values=(padval, padval))

            # tile the filters into an image
            data = data.reshape((n, n) + data.shape[1:]).transpose((0, 2, 1, 3) + tuple(range(4, data.ndim + 1)))
            data = data.reshape((n * data.shape[1], n * data.shape[3]) + data.shape[4:])
            return data

        img = vis_square(generated_images)
        if not os.path.isdir('images_sgan_cifar10'):
            os.mkdir('images_sgan_cifar10')
        Image.fromarray(img).save(
            "images_sgan_cifar10/epoch_%d_step_%d.png" % (epoch, global_step))


if __name__ == '__main__':
    sgan = SGAN()
    sgan.train(epochs=50, batch_size=64, sample_interval=50)



Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 16, 16, 32)        896       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 64)          18496     
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 8, 8, 64)          0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 8, 8, 64)          0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 8, 8, 64)          256       
__________

/home/imi432_006/anaconda3/envs/tf/lib/python3.5/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


epoch:0 step:1[D loss: 0.669468, acc: 41.41%, op_acc: 3.91%] [G loss: 0.629763]
epoch:0 step:2[D loss: 0.577821, acc: 53.91%, op_acc: 10.16%] [G loss: 0.664819]
epoch:0 step:3[D loss: 0.589353, acc: 56.25%, op_acc: 4.69%] [G loss: 0.778054]
epoch:0 step:4[D loss: 0.610846, acc: 50.00%, op_acc: 7.81%] [G loss: 0.693027]
epoch:0 step:5[D loss: 0.604125, acc: 50.00%, op_acc: 3.91%] [G loss: 0.770279]
epoch:0 step:6[D loss: 0.584484, acc: 53.12%, op_acc: 7.03%] [G loss: 0.859444]
epoch:0 step:7[D loss: 0.561575, acc: 59.38%, op_acc: 8.59%] [G loss: 0.848159]
epoch:0 step:8[D loss: 0.536508, acc: 63.28%, op_acc: 13.28%] [G loss: 0.932814]
epoch:0 step:9[D loss: 0.536352, acc: 67.19%, op_acc: 15.62%] [G loss: 0.995789]
epoch:0 step:10[D loss: 0.543748, acc: 58.59%, op_acc: 15.62%] [G loss: 0.802022]
epoch:0 step:11[D loss: 0.541396, acc: 58.59%, op_acc: 15.62%] [G loss: 0.806490]
epoch:0 step:12[D loss: 0.545454, acc: 60.94%, op_acc: 10.94%] [G loss: 0.846594]
epoch:0 step:13[D loss: 0.62131

epoch:0 step:105[D loss: 0.443722, acc: 74.22%, op_acc: 20.31%] [G loss: 1.407029]
epoch:0 step:106[D loss: 0.413394, acc: 73.44%, op_acc: 30.47%] [G loss: 1.461171]
epoch:0 step:107[D loss: 0.398466, acc: 81.25%, op_acc: 24.22%] [G loss: 1.393965]
epoch:0 step:108[D loss: 0.422165, acc: 78.12%, op_acc: 18.75%] [G loss: 1.509495]
epoch:0 step:109[D loss: 0.422917, acc: 83.59%, op_acc: 20.31%] [G loss: 1.340097]
epoch:0 step:110[D loss: 0.411540, acc: 79.69%, op_acc: 22.66%] [G loss: 1.374233]
epoch:0 step:111[D loss: 0.401864, acc: 79.69%, op_acc: 24.22%] [G loss: 1.723833]
epoch:0 step:112[D loss: 0.398693, acc: 76.56%, op_acc: 26.56%] [G loss: 1.433453]
epoch:0 step:113[D loss: 0.390089, acc: 78.12%, op_acc: 22.66%] [G loss: 1.614495]
epoch:0 step:114[D loss: 0.336757, acc: 86.72%, op_acc: 31.25%] [G loss: 1.707093]
epoch:0 step:115[D loss: 0.405968, acc: 78.91%, op_acc: 28.12%] [G loss: 1.602411]
epoch:0 step:116[D loss: 0.389629, acc: 79.69%, op_acc: 34.38%] [G loss: 1.496723]
epoc

...

epoch:49 step:38632[D loss: 0.421543, acc: 63.28%, op_acc: 40.62%] [G loss: 0.952029]
epoch:49 step:38633[D loss: 0.420855, acc: 61.72%, op_acc: 36.72%] [G loss: 0.892639]
epoch:49 step:38634[D loss: 0.392364, acc: 64.84%, op_acc: 39.84%] [G loss: 0.960168]
epoch:49 step:38635[D loss: 0.408147, acc: 54.69%, op_acc: 46.88%] [G loss: 0.945009]
epoch:49 step:38636[D loss: 0.432080, acc: 60.16%, op_acc: 40.62%] [G loss: 0.868516]
epoch:49 step:38637[D loss: 0.375933, acc: 71.09%, op_acc: 47.66%] [G loss: 0.866539]
epoch:49 step:38638[D loss: 0.398085, acc: 63.28%, op_acc: 45.31%] [G loss: 0.878376]
epoch:49 step:38639[D loss: 0.430556, acc: 52.34%, op_acc: 44.53%] [G loss: 0.900306]
epoch:49 step:38640[D loss: 0.422720, acc: 57.81%, op_acc: 39.06%] [G loss: 0.798629]
epoch:49 step:38641[D loss: 0.437603, acc: 53.12%, op_acc: 42.19%] [G loss: 0.945847]
epoch:49 step:38642[D loss: 0.391786, acc: 68.75%, op_acc: 42.19%] [G loss: 0.857551]
epoch:49 step:38643[D loss: 0.379691, acc: 67.97%, op_

epoch:49 step:38729[D loss: 0.409387, acc: 56.25%, op_acc: 49.22%] [G loss: 0.876017]
epoch:49 step:38730[D loss: 0.416250, acc: 56.25%, op_acc: 40.62%] [G loss: 0.884438]
epoch:49 step:38731[D loss: 0.418922, acc: 52.34%, op_acc: 45.31%] [G loss: 0.967304]
epoch:49 step:38732[D loss: 0.387767, acc: 63.28%, op_acc: 46.09%] [G loss: 0.891643]
epoch:49 step:38733[D loss: 0.426457, acc: 60.94%, op_acc: 42.19%] [G loss: 0.963697]
epoch:49 step:38734[D loss: 0.409588, acc: 51.56%, op_acc: 45.31%] [G loss: 0.929082]
epoch:49 step:38735[D loss: 0.405090, acc: 60.16%, op_acc: 42.19%] [G loss: 0.866547]
epoch:49 step:38736[D loss: 0.403014, acc: 57.81%, op_acc: 45.31%] [G loss: 0.809305]
epoch:49 step:38737[D loss: 0.409533, acc: 57.81%, op_acc: 42.97%] [G loss: 0.845811]
epoch:49 step:38738[D loss: 0.370704, acc: 67.97%, op_acc: 45.31%] [G loss: 0.966799]
epoch:49 step:38739[D loss: 0.418414, acc: 55.47%, op_acc: 45.31%] [G loss: 0.947454]
epoch:49 step:38740[D loss: 0.416730, acc: 66.41%, op_

epoch:49 step:38827[D loss: 0.376451, acc: 64.84%, op_acc: 50.78%] [G loss: 0.930822]
epoch:49 step:38828[D loss: 0.392965, acc: 63.28%, op_acc: 46.09%] [G loss: 0.940440]
epoch:49 step:38829[D loss: 0.437829, acc: 58.59%, op_acc: 45.31%] [G loss: 0.938960]
epoch:49 step:38830[D loss: 0.400672, acc: 64.84%, op_acc: 46.88%] [G loss: 0.867798]
epoch:49 step:38831[D loss: 0.440772, acc: 50.00%, op_acc: 36.72%] [G loss: 0.877000]
epoch:49 step:38832[D loss: 0.394464, acc: 73.44%, op_acc: 39.06%] [G loss: 0.988083]
epoch:49 step:38833[D loss: 0.378318, acc: 66.41%, op_acc: 47.66%] [G loss: 0.842974]
epoch:49 step:38834[D loss: 0.397558, acc: 62.50%, op_acc: 42.97%] [G loss: 0.961189]
epoch:49 step:38835[D loss: 0.423201, acc: 60.94%, op_acc: 42.97%] [G loss: 0.878769]
epoch:49 step:38836[D loss: 0.419863, acc: 57.81%, op_acc: 41.41%] [G loss: 0.924094]
epoch:49 step:38837[D loss: 0.368856, acc: 70.31%, op_acc: 46.88%] [G loss: 0.875959]
epoch:49 step:38838[D loss: 0.422696, acc: 54.69%, op_

epoch:49 step:38924[D loss: 0.410809, acc: 56.25%, op_acc: 44.53%] [G loss: 0.917549]
epoch:49 step:38925[D loss: 0.436339, acc: 60.16%, op_acc: 39.84%] [G loss: 0.891720]
epoch:49 step:38926[D loss: 0.402101, acc: 64.06%, op_acc: 42.19%] [G loss: 0.999726]
epoch:49 step:38927[D loss: 0.405738, acc: 65.62%, op_acc: 41.41%] [G loss: 0.938858]
epoch:49 step:38928[D loss: 0.404720, acc: 67.97%, op_acc: 43.75%] [G loss: 1.008094]
epoch:49 step:38929[D loss: 0.397073, acc: 61.72%, op_acc: 48.44%] [G loss: 0.976709]
epoch:49 step:38930[D loss: 0.369306, acc: 67.97%, op_acc: 49.22%] [G loss: 1.015913]
epoch:49 step:38931[D loss: 0.401182, acc: 59.38%, op_acc: 47.66%] [G loss: 0.871060]
epoch:49 step:38932[D loss: 0.405588, acc: 62.50%, op_acc: 48.44%] [G loss: 0.880602]
epoch:49 step:38933[D loss: 0.418855, acc: 56.25%, op_acc: 43.75%] [G loss: 0.830391]
epoch:49 step:38934[D loss: 0.408847, acc: 66.41%, op_acc: 46.88%] [G loss: 0.915070]
epoch:49 step:38935[D loss: 0.399601, acc: 69.53%, op_

epoch:49 step:39021[D loss: 0.419979, acc: 57.03%, op_acc: 45.31%] [G loss: 0.994147]
epoch:49 step:39022[D loss: 0.380792, acc: 64.06%, op_acc: 49.22%] [G loss: 0.922682]
epoch:49 step:39023[D loss: 0.419175, acc: 61.72%, op_acc: 39.84%] [G loss: 0.927596]
epoch:49 step:39024[D loss: 0.391314, acc: 72.66%, op_acc: 44.53%] [G loss: 0.878798]
epoch:49 step:39025[D loss: 0.399847, acc: 56.25%, op_acc: 46.09%] [G loss: 1.013350]
epoch:49 step:39026[D loss: 0.389477, acc: 69.53%, op_acc: 46.09%] [G loss: 0.925068]
epoch:49 step:39027[D loss: 0.377138, acc: 66.41%, op_acc: 50.00%] [G loss: 0.987106]
epoch:49 step:39028[D loss: 0.389431, acc: 63.28%, op_acc: 41.41%] [G loss: 0.876758]
epoch:49 step:39029[D loss: 0.399209, acc: 63.28%, op_acc: 46.88%] [G loss: 0.970980]
epoch:49 step:39030[D loss: 0.396900, acc: 67.19%, op_acc: 46.88%] [G loss: 0.845573]
epoch:49 step:39031[D loss: 0.449135, acc: 46.09%, op_acc: 39.06%] [G loss: 0.876428]
epoch:49 step:39032[D loss: 0.433274, acc: 59.38%, op_